# Тестовое задание

Требуется разработать алгоритм максимизации функции k на ориентированном мультиграфе (ССиТ).

Построение алгоритма будет выполняться в 2 этапа:
1. Написание движка для нахождения функции k на заданном ориентированном мультиграфе (ССиТ)
2. Оптимизация функции k

### Этап 1: Написание движка для нахождения функции k на заданном ориентированном мультиграфе

In [42]:

from sympy import symbols, Eq, solve

class Well:
    def __init__(self, idx, alpha, const, Q):
        self.idx = idx
        self.alpha = alpha
        self.const = const
        # self.edge_input = edge_input
        self.Q = Q
    # def flow_rate(self):
    #     return self.alpha * (  ) * self.const
          
class Vertex:
    def __init__(self, idx):
        self.idx = idx
        self.edge_input = []
        # self.edge_input = edge_input # List of pipes that are included in the node
        self.Q = 0

    def update_Q(self):
        self.Q = sum(edge.Q_edge for edge in self.edge_input)

class Edge:
    def __init__(self, idx, vertex1, vertex2, length = 5, diameter = 2, roughness = 0.1, density = 1000):
        self.idx = idx
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.length = length
        self.diameter = diameter
        self.roughness = roughness
        self.density = density
        self.update_Q_edge()

    def update_Q_edge(self):
        self.Q_edge = self.vertex1.Q
        self.pressure_loss = self.density * self.length * self.Q_edge * self.roughness / self.diameter

class Drain:
    def __init__(self, p_0, alpha = 2, const = 10, idx=0):
        self.p_0 = p_0
        self.edge_input = []
        self.alpha = alpha
        self.const = const
        self.idx = idx
        self.Q_0 = self.alpha * self.p_0 + self.const

In [47]:
# Body of the algorithm
alpha = 3
const = 5
density = 1000
def create_graph():
    # get the number of Wells
    num_wells = int(input("Enter the number of Wells: "))
    wells = []
    
    # Создание списка из num_wells элементов - Q1, Q2, Q3
    flow_rate = []
    for i in range(num_wells):
        flow_rate.append('Q' + str(i+1))
    flow_rate = symbols(flow_rate)
    
    counter = 0
    for i in range(num_wells):
        counter = counter + 1
        wells.append(Well(i+1, alpha, const, flow_rate[i]))

    # get the number of Vertex
    num_vertex = int(input("Enter the number of Vertex: "))
    vertex = []
    for i in range(num_vertex):
        vertex.append(Vertex(i + counter))

    # Create Drain
    drain = [Drain(100)]
    common_vertex = drain + wells + vertex # A common array of all vertices
    print(common_vertex)

    # get the number of Edges
    num_edges = int(input("Enter the number of Edges: "))
    edges = []
    for i in range(num_edges):
        vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1)))
        vertex2 = int(input("Enter the second vertex for edge {}: ".format(i + 1)))
        # length = float(input("Enter the length for edge {}: ".format(i + 1)))
        # diameter = float(input("Enter the diameter for edge {}: ".format(i + 1)))
        # roughness = float(input("Enter the roughness for edge {}: ".format(i + 1)))
        # edges.append(Edge(i+1, common_vertex[vertex1], common_vertex[vertex2], length, diameter, roughness))
        edges.append(Edge(i+1, common_vertex[vertex1], common_vertex[vertex2]))

    for edge in edges:
        # edge.vertex1.edge_input.append(edge)
        edge.vertex2.edge_input.append(edge)

    while edges[num_edges-1].Q_edge == 0:
        # for well in wells:
        #     well.flow_rate()

        for edge in edges:
            edge.update_Q_edge()

        for vert in vertex:
            vert.update_Q()

    return wells, edges

In [48]:
wells, edges = create_graph()

[<__main__.Drain object at 0x00000206E941C730>, <__main__.Well object at 0x00000206E9322B50>, <__main__.Well object at 0x00000206E941CC70>, <__main__.Well object at 0x00000206E941CE80>, <__main__.Well object at 0x00000206E941CEB0>, <__main__.Vertex object at 0x00000206E941C370>, <__main__.Vertex object at 0x00000206E941CCD0>, <__main__.Vertex object at 0x00000206E941C910>, <__main__.Vertex object at 0x00000206E941C0D0>]


In [49]:
# print(edges[0].Q_edge)
# print(edges[1].Q_edge)
# print(edges[2].Q_edge)
# print(edges[3].Q_edge)
# print(edges[4].Q_edge)
# print(edges[7].Q_edge)
# print(edges[7].pressure_loss)
# print(type(edges[0].Q_edge))

Q1 + Q2
Q1 + Q2 + Q3 + Q4
250.0*Q1 + 250.0*Q2 + 250.0*Q3 + 250.0*Q4
<class 'sympy.core.symbol.Symbol'>


In [ ]:
def bfs(wells, edges, drain):
queue = []
visited = set()
parent = {}
paths = {}

## Тестовая часть, понадлежит удалению перед отправкой

In [7]:
from sympy import symbols, Eq, solve

x, y = symbols('x y')

class MyExpression:
    def __init__(self):
        self.expr = x + y
        
    def evaluate(self, x_value, y_value):
        return self.expr.subs([(x, x_value), (y, y_value)])

def evaluate_expression(expr, x_value, y_value):
    return expr.subs([(x, x_value), (y, y_value)])

expr = MyExpression()
# print(evaluate_expression(expr, 2, 3)) # Output: 5
print(expr.evaluate(2, 3)) # Output: 5

5


In [1]:
from sympy import symbols, Eq, solve


variables = ['x', 'y', 'z']
variables = symbols(variables)
print(variables)
print(type(variables[0]))

variables = ['x', 'y', 'z']
x, y, z = symbols(','.join(variables))
print(x, y, z)
print(type(x))


[x, y, z]
<class 'sympy.core.symbol.Symbol'>
x y z
<class 'sympy.core.symbol.Symbol'>


In [ ]:
input().strip().split()
print(a)

In [5]:
b = []
for i in range(3):
        b.append('Q' + str(i))

print(b)

['Q0', 'Q1', 'Q2']


In [2]:
i = 1
vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1)))
print(vertex1)

1


In [21]:
drain = [Drain(100)]
print(drain[0].idx)

0


In [26]:
a = []
a.append(1)
print(a)

[1]
